In [1]:
import pandas as pd

# Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:

- оценка 2 и ниже — низкий рейтинг;
- оценка 4 и ниже — средний рейтинг;
- оценка 4.5 и 5 — высокий рейтинг.

Результат классификации запишите в столбец class.

In [2]:
movies = pd.read_csv('movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [3]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
def class_movies(rating):
    if rating is None:
        return None
    elif rating <= 2:
        return 'Низкий рейтинг'
    elif rating >= 4.5:
        return 'Высокий рейтинг'
    else:
        return 'Средний рейтинг'

In [5]:
movies['class'] = movies.merge(
    ratings.groupby('movieId').rating.mean(),
    how='left',
    on='movieId'
)['rating'].apply(class_movies)
movies.head()

,movieId,title,genres,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Средний рейтинг
1,2,Jumanji (1995),Adventure|Children|Fantasy,Средний рейтинг
2,3,Grumpier Old Men (1995),Comedy|Romance,Средний рейтинг
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Средний рейтинг
4,5,Father of the Bride Part II (1995),Comedy,Средний рейтинг


# Задание 2
Используйте файл keywords.csv.

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
```
geo_data = {


'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```
Результат классификации запишите в отдельный столбец region.

In [6]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [7]:
# Мне показалось логичным перестроить словарь для ускорения поиска
regions = dict()
for region, cities in geo_data.items():
    for city in cities:
        regions[city] = region
    
regions

{'москва': 'Центр',
 'тула': 'Центр',
 'ярославль': 'Центр',
 'петербург': 'Северо-Запад',
 'псков': 'Северо-Запад',
 'мурманск': 'Северо-Запад',
 'владивосток': 'Дальний Восток',
 'сахалин': 'Дальний Восток',
 'хабаровск': 'Дальний Восток'}

In [8]:
def get_region(keyword):
    for word in keyword.split():
        if word in regions:
            return regions[word]
    return 'undefined'

In [9]:
keywords = pd.read_csv('keywords.csv')
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [12]:
keywords['region'] = keywords['keyword'].apply(get_region)
keywords['region'].value_counts()

region
undefined         99406
Центр               384
Северо-Запад        146
Дальний Восток       64
Name: count, dtype: int64

# Задание 3 (бонусное)

Есть мнение, что раньше снимали настоящее кино, не то что сейчас. Ваша задача — проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия: файл movies.csv и ratings.csv из базы. Нужно проверить, верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

Вы не будете затрагивать субьективные факторы выставления этих рейтингов, а пройдётесь по алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010 года.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте алгоритм:
    1. для каждой строки пройдите по всем годам списка years;
    2. если номер года присутствует в названии фильма, то функция возвращает этот год, как год выпуска;
    3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год.
    

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’.

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга.

Все файлы находятся в архиве «Дополнительные файлы к лекции и дз».

In [38]:
years =[str(year) for year in range(1950, 2011)]
movies.head()

,movieId,title,genres,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Средний рейтинг
1,2,Jumanji (1995),Adventure|Children|Fantasy,Средний рейтинг
2,3,Grumpier Old Men (1995),Comedy|Romance,Средний рейтинг
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Средний рейтинг
4,5,Father of the Bride Part II (1995),Comedy,Средний рейтинг


In [43]:
def production_year(name):
    for year in years:
        if year in name:
            return year
    return '1900'

In [72]:
movies['year'] = movies['title'].apply(production_year)
movies.merge(
    ratings.groupby('movieId').rating.mean(),
    how='left',
    on='movieId'
).groupby('year')['rating'].mean().sort_values(ascending=False)

year
1971    3.811243
1957    3.767680
1951    3.718044
1952    3.700104
1955    3.672863
          ...   
1992    3.082417
1988    3.067250
1987    3.047063
1989    3.045879
1991    3.030564
Name: rating, Length: 62, dtype: float64

# Ускорение
Мне показалось, что предложенный вариант поиска года слишком медленный и попробовал свой.
Если я правильно все сделал, то он оказался быстрее

In [47]:
%%timeit

movies['title'].apply(production_year)

12 ms ± 143 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [51]:
import regex as re
pattern = re.compile(r'\d{4}')


In [75]:
def production_year2(name):
    for year in pattern.findall(name):
        if 1950 <= int(year) <= 2010:
            return year
    return '1900'

In [76]:
%%timeit

movies['title'].apply(production_year2)

4.8 ms ± 18.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
